## Timothy Miller
## GTECH 73100, Dr. Sun

# [Assignment Three](https://github.com/TangoYankee/gtech_731-geocomp-hw/tree/main/assignment-three)
Adjustments to assignment two

## Modifications
I considered several changes to the previous implementation. First, I examined the list and dictionary construction functions to determine whether any of them would be easy to read and more concise as comprehensions. I decided that only `get_county_totals` would be improved with a list comprehension. Next, I considered whether any of the dictionary constructions in task three would benefit from consolidation into a single constructor. For example, the first and third parts both require counting the total counties in each state. However, I decided that keeping them as separate functions made them easier to understand and maintain- this is worth a little bit of duplication. I also considered reusuing mock data between several tests. However, I once again decided that keeping these tests independent of each other was worth a bit of code duplication. 

Finally, I reexamined the use of the built-in sort function when sorting the rankings. The rankings list will only ever have one element out of position. I wanted to check whether this knowledge could lead to a more performant custom sorting algorithm that takes advantage of this structure. I created a 'simple bubble' function to sort the rankings. I then compared its performance to the builtin sort. It's worse. But, now I know.

I only included the portions of code with meaningful changes, to help make them easier to read. Ultimately, these changes were limited to a few parts of Task two. Additionally, there are comments noting the modifications.

### Import modules

In [3]:
import json
import io
import time

### Read in a data file of all counties in the US.  

In [4]:
# Data Source https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_050_00_5m.json
# address utf-8 error https://stackoverflow.com/questions/30996289/utf8-codec-cant-decode-byte-0xf3
# Use "with ... as ..." to better handle exceptions
with io.open('data/gz_2010_us_050_00_5m.json', encoding='latin-1') as f:
  data = json.load(f)

features = data['features']

### Task 2

Derive the numbers of counties that use these three names, respectively. For each of them, list their county name and state code.

In [6]:
def get_county_states(features):
  """Format the counties to easily list their states

  Arguments:
  features list[dict] -- county data

  Returns:
  dict[str, list[str]] -- County name as the key and the list of state codes as the value
  """
  county_states = dict()
  for feature in features:
    properties = feature['properties']
    county_name = properties['NAME']
    state_code = properties['STATE']
    county_states.setdefault(county_name, []).append(state_code)

  return county_states


def test_get_county_states():
  mock_features = [{
    'type': 'Feature',
    "properties": {
      'GEO_ID': '0500000US01087',
      'STATE': '01',
      'COUNTY': '087',
      'NAME': 'Macon',
      'LSAD': 'County',
      'CENSUSAREA': 608.885
    },
    "geometry": None,
  }, {
    'type': 'Feature',
    "properties": {
      'GEO_ID': '0500000US02275',
      'STATE': '02',
      'COUNTY': '275',
      'NAME': 'Wrangell',
      'LSAD': 'Cty&Bor',
      'CENSUSAREA': 2541.483
    },
    "geometry": None,
  }, {
    'type': 'Feature',
      'properties': {
      'GEO_ID': '0500000US02270',
      'STATE': '02',
      'COUNTY': '270',
      'NAME': 'Wade Hampton',
      'LSAD': 'CA',
      'CENSUSAREA': 17081.433
    },
    "geometry": None,
  }, {
    'type': 'Feature',
      'properties': {
      'GEO_ID': '',
      'STATE': '03',
      'COUNTY': '',
      'NAME': 'Wade Hampton',
      'LSAD': '',
      'CENSUSAREA': 0
    },
    "geometry": None,
  }]

  expected = {
    "Macon": ["01"],
    "Wrangell": ["02"],
    "Wade Hampton": ["02", "03"]
  }

  obs = get_county_states(mock_features)
  assert(obs == expected)

test_get_county_states()

county_states = get_county_states(features)

def get_county_totals(county_states):
  """Reformat the counties and state list objects into a tuple of counties and their totals

  Arguments:
  county_states dict[str, list[str]] -- County name as the key and the list of state codes as the value

  Returns:
  tuple[str, int] -- The counties and the total number of states that use them.
  """
  # Modified to use list comprehension
  return [(county, len(states)) for county, states in county_states.items()]


def test_get_county_totals():
  mock_county_states = {
    "A": ['0'],
    "B": ['0', '1'],
    "C": ['0', '1', '2']
  }
  expected_count_totals = [('A', 1), ('B', 2), ('C', 3)]
  assert(get_county_totals(mock_county_states) == expected_count_totals)

test_get_county_totals()

# Modification: create simple bubble sort 
def simple_bubble(totals):
  """Sorting based on the assumption that only the first item needs to be 'bubbled up' to its correct space

  Arguments:
  totals list[tuple[str, int]] -- Item Name and total 
  
  Returns:
  None: list is mutated in place 
  """

  target_pos = 0
  compare_pos = 1
  totals_count = len(totals)
  while compare_pos < totals_count and totals[target_pos][1] > totals[compare_pos][1]:
    totals[target_pos], totals[compare_pos] = totals[compare_pos], totals[target_pos]
    target_pos += 1
    compare_pos += 1

def test_simple_bubble():
    mock_totals = [("C", 5), ("B", 0), ("A", 5)]
    expected_totals = [("B", 0), ("C", 5), ("A", 5)]

    # Sort in place
    simple_bubble(mock_totals) 

    assert(mock_totals == expected_totals)

test_simple_bubble() 

# Modification: create function that sorts with a simple bubble sort
def top_k_sort_k_bubble(totals, k):
  """Find the top k values in a tuple of objects and their counts.
  
  Arguments:
  totals tuple[str, int]-- Item Name and total 
  k int -- the number of items to rank

  Returns:
  tuple[str, int] -- Top k items and their counts
  
  Note:
  This function iterates through the list of items, only sorting the list of rankings
  """
  top_k = [(None, 0)]*k
  for total in totals:
    total_val = total[1]
    bottom_k_val = top_k[0][1]
    if total_val > bottom_k_val:
      top_k[0] = total
      simple_bubble(top_k)

  return top_k

def test_top_k_sort_k_simple():
  mock_county_totals = [("A", 0), ("B", 1), ("C", 2), ("D", 3), ("E", 4), ("F", 4)]
  expected_top_k = [("D", 3), ("F", 4), ("E", 4)]

  obs = top_k_sort_k_bubble(mock_county_totals, 3)

  assert(obs == expected_top_k)

test_top_k_sort_k_simple()

def top_k_sort_k(totals, k):
  """Find the top k values in a tuple of objects and their counts.
  
  Arguments:
  totals tuple[str, int]-- Item Name and total 
  k int -- the number of items to rank

  Returns:
  tuple[str, int] -- Top k items and their counts
  
  Note:
  This function iterates through the list of items, only sorting the list of rankings
  """
  top_k = [(None, 0)]*k
  for total in totals:
    total_val = total[1]
    bottom_k_val = top_k[0][1]
    if total_val > bottom_k_val:
      top_k[0] = total
      top_k.sort(key=lambda a: a[1])

  return top_k

def test_top_k_sort_k():
  mock_county_totals = [("A", 0), ("B", 1), ("C", 2), ("D", 3), ("E", 4), ("F", 4)]
  expected_top_k = [("D", 3), ("F", 4), ("E", 4)]
  
  assert(top_k_sort_k(mock_county_totals, 3) == expected_top_k)

test_top_k_sort_k()

county_totals = get_county_totals(county_states)
# Modification: Compare the previous sort of the rankings with the 
# new method that uses a simple bubble
"""
Compare the built-in and simple bumble sort algorithm approaches

Findings: Built in sort is almost always faster
"""
k = 200 
start_time = time.perf_counter()
top_counties = top_k_sort_k(county_totals, k)
stop_time = time.perf_counter()
print(f"top counties built in: {top_counties}")
print(f"top counter built in time: {stop_time - start_time}")

start_time = time.perf_counter()
top_counties_bubble = top_k_sort_k_bubble(county_totals, k)
stop_time = time.perf_counter()
print(f"top counties bubble: {top_counties_bubble}")
print(f"top counter bubble time: {stop_time - start_time}")

top counties built in: [('Vernon', 3), ('Choctaw', 3), ('Ohio', 3), ('Lowndes', 3), ('Murray', 3), ('Cheyenne', 3), ('Hall', 3), ('Buchanan', 3), ('Grayson', 3), ('Gallatin', 3), ('Delta', 3), ('Pawnee', 3), ('Morris', 3), ('Meade', 3), ('Daviess', 3), ('Butte', 3), ('Holmes', 3), ('Seminole', 3), ('Summit', 3), ('Park', 3), ('Baker', 3), ('Sussex', 3), ('Miller', 3), ('Stone', 3), ('Sevier', 3), ('Platte', 3), ('Chippewa', 3), ('Williamson', 3), ('Moore', 3), ('Haskell', 3), ('Fairfield', 3), ('Chester', 3), ('Wyoming', 3), ('Cameron', 3), ('Beaver', 3), ('Stephens', 3), ('McIntosh', 3), ('Potter', 3), ('Buffalo', 3), ('Orleans', 3), ('Oneida', 3), ('Miami', 3), ('Burke', 3), ('Schuyler', 3), ('Christian', 3), ('Lauderdale', 3), ('Rock', 3), ('McPherson', 3), ('Covington', 3), ('Todd', 3), ('St. Louis', 3), ('Pope', 3), ('Osceola', 3), ('Thomas', 3), ('Stevens', 3), ('Osage', 3), ('Edwards', 3), ('Dickinson', 3), ('Comanche', 3), ('Wright', 3), ('Worth', 3), ('Humboldt', 3), ('Claibor